In [1]:
import pandas as pd
import numpy as np
import requests
import pendulum
from datetime import tzinfo, timedelta, datetime, timezone, time
import os

In [2]:
# import required module

def isImageFilePresent(directory, file_name):
    file_present = os.path.isfile(file_name)
    if file_present:
        print('Alreaday present : '+file_name)

    return file_present

In [3]:
def download_image_url(date, day_night_ind, bbox, latitude, longitude):
    """
    example arguments:
     time: datetime(2020-01-29T12:10:00Z)
     bbox: [10.177734375, -60.1875, 16.41796875, -53.5166015625]
     layers: ["GOES-East_ABI_Band2_Red_Visible_1km", "Reference_Labels"]
     image_format: "tiff"
     resolution: 0.01 (deg/pixel)

     `bbox` should be a list of [lat_S, lon_W, lat_N, lon_E]
     
    """
    BASE_URL = "https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&LAYERS=MODIS_Aqua_CorrectedReflectance_TrueColor,MODIS_Aqua_Thermal_Anomalies_Day&CRS=EPSG:4326&TIME={datetimestamp}&WRAP=DAY,DAY&BBOX={bbox}&FORMAT=image/png&WIDTH=5670&HEIGHT=4403&AUTOSCALE=TRUE&ts={timestamp}"

    date_split = date.split("-")
    year = date_split[0]
    month = date_split[1]
    day = date_split[2]

    hour = 12
    if day_night_ind == "N":
        hour = 22
    
    dt = datetime(int(year), int(month), int(day), hour, 0, 0, 0, tzinfo=timezone.utc)
    #dt = pendulum.instance(dt).to_iso8601_string()
    print(dt)
    

    url = BASE_URL.format(
        datetimestamp=date,
        bbox=",".join([str(v) for v in bbox]),
        timestamp = datetime.timestamp(dt)
    )

    print("url : "+url)
    return url
    

def download_img(date, day_night_ind, bbox, latitude, longitude):
    url = download_image_url(date, day_night_ind, bbox, latitude, longitude)
    date_split = date.split("-")
    year = date_split[0]
    month = date_split[1]
    day = date_split[2]
    file_name = "./satellite_image/"+year+'/'+str(longitude)+'_'+str(latitude)+'-'+date+'-'+day_night_ind+'.png'
    directory = "./satellite_image/"+year
    #print(url)
    if not isImageFilePresent(directory, file_name):
        r = requests.get(url)
        if r.status_code == 200:
            if 'xml' in r.text[:40]:
                print("File not downloaded : "+url)
            else:
                with open(file_name, 'wb') as fh:
                    fh.write(r.content)
        else:
            print("File not downloaded : not 200"+url)



In [4]:
def group_fire_data_xarray(file_name):
    file_path = "./historical_fire_data/" + file_name + ".csv"
    fire_data = pd.read_csv(file_path)
    
    filtered_fire_data = fire_data.query("`confidence` >= 75 and `type` == 0")
    grouped_fire_data = filtered_fire_data.groupby(['acq_date', 'daynight']).agg({'latitude':'mean','longitude':'mean','brightness':'mean','confidence':'mean', 'frp':'mean', 'bright_t31':'mean'}).reset_index()

    one_downloaded = 0
    for modis_data_index, modis_row in grouped_fire_data.iterrows():
        
        date_value = modis_row['acq_date']
        confidence = modis_row['confidence']
        latitude = modis_row['latitude']
        longitude = modis_row['longitude']
        day_night_ind = modis_row['daynight']
        #print(type(confidence))
        if confidence > 75:
            north_coord = latitude+5
            south_coord = latitude-5
            west_coord = longitude-5
            east_coord = longitude+5
            area = [south_coord, west_coord, north_coord, east_coord]
            #area = [44.1105,-80.8925,63.4595,-55.9775]
            download_img(date_value, day_night_ind, area, latitude, longitude )


In [5]:
group_fire_data_xarray("canada_2022")

2022-01-11 12:00:00+00:00
url : https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&LAYERS=MODIS_Aqua_CorrectedReflectance_TrueColor,MODIS_Aqua_Thermal_Anomalies_Day&CRS=EPSG:4326&TIME=2022-01-11&WRAP=DAY,DAY&BBOX=50.339499999999994,-125.7944,60.339499999999994,-115.7944&FORMAT=image/png&WIDTH=5670&HEIGHT=4403&AUTOSCALE=TRUE&ts=1641902400.0
Alreaday present : ./satellite_image/2022/-120.7944_55.339499999999994-2022-01-11-D.png
2022-01-12 12:00:00+00:00
url : https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&LAYERS=MODIS_Aqua_CorrectedReflectance_TrueColor,MODIS_Aqua_Thermal_Anomalies_Day&CRS=EPSG:4326&TIME=2022-01-12&WRAP=DAY,DAY&BBOX=49.6095,-120.60915,59.6095,-110.60915&FORMAT=image/png&WIDTH=5670&HEIGHT=4403&AUTOSCALE=TRUE&ts=1641988800.0
Alreaday present : ./satellite_image/2022/-115.60915_54.6095-2022-01-12-D.png
2022-01-13 12:00:00+00:00
url : https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&LAYERS=MODIS_Aqua_CorrectedReflectance_